In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import mpld3
import os
import webbrowser
import matplotlib
import mplcursors
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm 
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import scipy.stats as stats
import networkx as nx
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from pychow import chow_test
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.api import OLS
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup
import requests
from lxml import html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import scipy.stats as stats
import networkx as nx
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from pychow import chow_test
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.api as smt
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.stattools import jarque_bera
from arch import arch_model
from statsmodels.stats.diagnostic import het_white, het_breuschpagan, het_goldfeldquandt, het_arch
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import streamlit as st
import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox
import pandas as pd
from IPython.display import display

In [3]:
df = pd.read_excel('data_01_socio_economic_102_v20240607.xlsx')

In [4]:
df = df[df['indicator_name'] != "Смертность населения (без показателя смертности от внешних причин)"]

In [8]:
dfs = []

for name in df['indicator_name'].unique():
    df1 = df[df['indicator_name'] == name]
    df1 = df1[['indicator_name', 'object_name', 'object_level', 'year', 'indicator_value', 'indicator_unit']]
    assert(all([df1['year'].value_counts().values[i] == len(df1['object_name'].unique()) for i in range(len(df1['year'].value_counts().values))]))
    assert(len(df1['indicator_unit'].unique()) == 1)
    dfs.append(df1)

In [9]:
df = pd.concat(dfs, axis=0)

In [10]:
# все пропуски помечены как np.nan
df['indicator_value'] = df['indicator_value'].where(df['indicator_value'] != -99999999, np.nan)
df['indicator_value'] = df['indicator_value'].where(df['indicator_value'] != -77777777, np.nan)

In [14]:
# инициализируем года
Население = {year : None for year in df['year'].unique()}

In [16]:
for key in Население.keys():
    current_df = df[df['year'] == key].copy()
    current_df.loc[:,'feature_name'] = current_df['indicator_name'] + ' (' + current_df['indicator_unit'] + ')'

    # собсвтенно генерация датасета для значения в словаре
    new_df = pd.DataFrame()
    new_df['object_name'] = current_df['object_name'].unique()
    new_df['object_level'] = pd.merge(left=new_df, right=current_df, on='object_name', how='right')['object_level']
    new_df['year'] = key
    for feature_name in sorted(current_df['feature_name'].unique()):
        new_df[feature_name] = pd.merge(left=new_df, right=current_df[current_df['feature_name'] == feature_name], on='object_name', how='right')['indicator_value']

    # проверка, что все регионы были включены 
    assert(len(new_df) == 96)
    Население[key] = new_df

total = set()
for key in Население.keys():
    total.update(set(Население[key].columns))

In [18]:
df = pd.read_excel('data_02_socio_economic_102_v20240607.xlsx')

In [19]:
needed_cols = np.array([k for k, v in df['indicator_name'].value_counts().items() if v == 2208])

In [20]:
df = df[df['indicator_name'].isin(needed_cols)]

In [21]:
dfs = []

for name in df['indicator_name'].unique():
    df1 = df[df['indicator_name'] == name]
    df1 = df1[['indicator_name', 'object_name', 'object_level', 'year', 'indicator_value', 'indicator_unit']]
    assert(all([df1['year'].value_counts().values[i] == len(df1['object_name'].unique()) for i in range(len(df1['year'].value_counts().values))]))
    assert(len(df1['indicator_unit'].unique()) == 1)
    dfs.append(df1)

In [22]:
df = pd.concat(dfs, axis=0)

In [23]:
# все пропуски помечены как np.nan
df['indicator_value'] = df['indicator_value'].where(df['indicator_value'] != -99999999, np.nan)
df['indicator_value'] = df['indicator_value'].where(df['indicator_value'] != -77777777, np.nan)

In [24]:
# инициализируем года
Труд = {year : None for year in df['year'].unique()}

In [25]:
for key in Труд.keys():
    current_df = df[df['year'] == key].copy()
    current_df.loc[:,'feature_name'] = current_df['indicator_name'] + ' (' + current_df['indicator_unit'] + ')'

    # собсвтенно генерация датасета для значения в словаре
    new_df = pd.DataFrame()
    new_df['object_name'] = current_df['object_name'].unique()
    new_df['object_level'] = pd.merge(left=new_df, right=current_df, on='object_name', how='right')['object_level']
    new_df['year'] = key
    for feature_name in sorted(current_df['feature_name'].unique()):
        new_df[feature_name] = pd.merge(left=new_df, right=current_df[current_df['feature_name'] == feature_name], on='object_name', how='right')['indicator_value']

    # проверка, что все регионы были включены 
    assert(len(new_df) == 96)
    Труд[key] = new_df

total = set()
for key in Труд.keys():
    total.update(set(Труд[key].columns))

In [36]:
df = pd.read_excel('data_04_socio_economic_102_v20240607.xlsx')

In [37]:
# оставляем только признаки реленвантные в 2000 году
needed = df[df['year'] == 2000]['indicator_name'].unique()
df = df[df['indicator_name'].isin(needed)]

In [38]:
# все пропуски помечены как np.nan
df['indicator_value'] = df['indicator_value'].where(df['indicator_value'] != -99999999, np.nan)
df['indicator_value'] = df['indicator_value'].where(df['indicator_value'] != -77777777, np.nan)

In [39]:
dfs = []

for name in df['indicator_name'].unique():
    df1 = df[df['indicator_name'] == name]
    if (len(df1['year'].unique()) == 23):
        df1 = df1[['indicator_name', 'object_name', 'year', 'indicator_value', 'indicator_unit']]
        df1['indicator_value'] = df1['indicator_value'].where(df1['indicator_value'] >= 0, np.nan)
        assert(sorted(df1['year'].unique()) == [i for i in range(2000, 2023, 1)])
        assert(len(df1['indicator_unit'].unique()) == 1)
        dfs.append(df1)

In [40]:
# инициализируем года
Образование = {year : None for year in df['year'].unique()}

In [41]:
for key in Образование.keys():
    current_df = df[df['year'] == key].copy()
    current_df.loc[:,'feature_name'] = current_df['indicator_name'] + ' (' + current_df['indicator_unit'] + ')'

    # собсвтенно генерация датасета для значения в словаре
    new_df = pd.DataFrame()
    new_df['object_name'] = current_df['object_name'].unique()
    new_df['object_level'] = pd.merge(left=new_df, right=current_df, on='object_name', how='right')['object_level']
    new_df['year'] = key
    for feature_name in sorted(current_df['feature_name'].unique()):
        new_df[feature_name] = pd.merge(left=new_df, right=current_df[current_df['feature_name'] == feature_name], on='object_name', how='right')['indicator_value']

    # проверка, что все регионы были включены 
    assert(len(new_df) == 96)
    Образование[key] = new_df

total = set()
for key in Образование.keys():
    total.update(set(Образование[key].columns))

In [48]:
# выкидываем пропуски
def drop_missing_values(df, reference_columns):
    df_cleaned = df.dropna()
    df_cleaned = df_cleaned[reference_columns]
    
    return df_cleaned

def normalized_dict_by_minmax(dict_of_years: dict[int, pd.DataFrame]):
    neutral = (
        'object_name',
        'object_level',
        'year'
    )

    normilized_dict = {}
    for year in dict_of_years.keys():
        normilized_dict[year] = dict_of_years[year].copy()
        for category in normilized_dict[year].columns:
            if category in neutral:
                continue
            normilized_dict[year][category] = (normilized_dict[year][category] - normilized_dict[year][category].min()) / (normilized_dict[year][category].max() - normilized_dict[year][category].min())
    return normilized_dict

def normalized_dict_by_minmax_by_base_year(dict_of_years: dict[int, pd.DataFrame], base_year=2010):
    min_values_by_features = {}
    max_values_by_features = {}
    neutral = (
        'object_name',
        'object_level',
        'year'
    )

    for category in dict_of_years[base_year].columns:
        if category in neutral:
            continue
        min_values_by_features[category] = dict_of_years[base_year][category].min()
        max_values_by_features[category] = dict_of_years[base_year][category].max()

    normilized_dict = {}
    for year in dict_of_years.keys():
        normilized_dict[year] = dict_of_years[year].copy()
        for category in normilized_dict[year].columns:
            if category in neutral:
                continue
            normilized_dict[year][category] = (normilized_dict[year][category] - min_values_by_features[category]) / (max_values_by_features[category] - min_values_by_features[category])
    return normilized_dict

def calculate_index_with_weights(dict_of_dataframes, way_of_calculating=1):
    '''
    Вычисляет индекс на основе заданных данных и выбранного метода расчета весов.

    Параметры:
    dict_of_dataframes (dict): Словарь, где ключи - годы, а значения - DataFrame с данными.
    way_of_calculating (int): Метод расчета весов:
        1 - PCA (анализ главных компонент);
        2 - Обратная дисперсия;
        3 - Равные веса.

    Возвращает:
    tuple: Кортеж, содержащий DataFrame с индексами по годам и массив весов.
    '''
    
    indexed_data = {}
    combined_data = pd.concat(dict_of_dataframes.values())
    numerical_columns = combined_data.drop(columns=['object_name', 'object_level', 'year'])

    if way_of_calculating == 1:  # PCA
        pca = PCA(n_components=1)
        pca.fit(numerical_columns)
        weights = np.abs(pca.components_[0])
        weights /= weights.sum()
    
    elif way_of_calculating == 2:  # Обратная дисперсия
        variances = numerical_columns.var()
        weights = 1 / variances
        weights /= weights.sum()
    
    else:  # Равные веса
        num_columns = numerical_columns.shape[1]
        weights = np.ones(num_columns) / num_columns
    
    for year, df in dict_of_dataframes.items():
        numerical_columns = df.drop(columns=['object_name', 'object_level', 'year'])
        weighted_mean = (numerical_columns * weights).sum(axis=1)
        df['index'] = weighted_mean
        indexed_data[year] = df[['object_name', 'index']]
    
    combined_index = pd.DataFrame()
    for year, df in indexed_data.items():
        df = df.set_index('object_name')['index']
        combined_index[year] = df

    return combined_index, weights

def combine_indices(df1, df2, alpha):
    if not df1.shape == df2.shape:
        raise ValueError("Датафреймы должны иметь одинаковую структуру (размеры и индексы).")
    if not (df1.index.equals(df2.index) and df1.columns.equals(df2.columns)):
        raise ValueError("Датафреймы должны иметь одинаковые индексы и столбцы.")

    if not (0 <= alpha <= 1):
        raise ValueError("Alpha должен быть в пределах от 0 до 1.")

    combined_df = alpha * df1 + (1 - alpha) * df2
    return combined_df

def plot_index_trends_multi(dfs_by_base, dfs_by_current, dfs_final, region, domen):
    if not (dfs_by_base.index.equals(dfs_by_current.index) and dfs_by_current.index.equals(dfs_final.index)):
        raise ValueError("Индексы регионов должны совпадать во всех датасетах.")
    
    region_name = region
    base_index = dfs_by_base.loc[region]
    current_index = dfs_by_current.loc[region]
    final_index = dfs_final.loc[region]
    plt.figure(figsize=(14, 8))
    plt.plot(final_index.index, final_index.values, marker='o', linestyle='-', linewidth=3,
             label="Final Index", color='red', alpha=0.9)  # Основной индекс
    plt.plot(base_index.index, base_index.values, marker='o', linestyle='--', linewidth=2,
             label="Base Index", color='blue', alpha=0.5)
    plt.plot(current_index.index, current_index.values, marker='o', linestyle=':', linewidth=2,
             label="Current Index", color='green', alpha=0.5)
    plt.title(f"{domen}. Динамика индексов по годам для региона: {region_name}", fontsize=16, fontweight='bold')
    plt.xlabel("Год", fontsize=14)
    plt.ylabel("Индекс", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=12)
    for i in range(len(final_index)):
        plt.annotate(f"{final_index.values[i]:.2f}", 
                     (final_index.index[i], final_index.values[i]),
                     textcoords="offset points", 
                     xytext=(0,10), 
                     ha='center', fontsize=10, color='red')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.gca().set_facecolor('#f9f9f9')
    
    plt.show()

def plot_index_trends_with_similar(dfs_by_base, dfs_by_current, dfs_final, region_name, domen, k=3):
    if not (dfs_by_base.index.equals(dfs_by_current.index) and dfs_by_current.index.equals(dfs_final.index)):
        raise ValueError("Индексы регионов должны совпадать во всех датасетах.")
    if region_name not in dfs_final.index:
        raise ValueError(f"Регион '{region_name}' отсутствует в данных.")
    target_vector = dfs_final.loc[region_name].values.reshape(1, -1)
    distances = cdist(target_vector, dfs_final.values, metric='euclidean').flatten()
    similar_indices = np.argsort(distances)[1:k + 1]
    similar_regions = dfs_final.index[similar_indices]
    plt.figure(figsize=(14, 8))
    plt.gca().set_facecolor('white') 
    plt.grid(color='lightgrey', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.plot(dfs_final.columns, dfs_final.loc[region_name], 
             marker='o', linestyle='-', linewidth=3, label=f"{region_name} (Target)", 
             color='orange', alpha=0.9, zorder=5) 
    
    colors = ['#FF6347', '#4682B4', '#32CD32', '#FFD700', '#8A2BE2']
    
    for i, similar_region in enumerate(similar_regions):
        plt.plot(dfs_final.columns, dfs_final.loc[similar_region], 
                 marker='s', linestyle=':', linewidth=2, label=similar_region,
                 color=colors[i % len(colors)], alpha=0.8)

        max_value = dfs_final.loc[similar_region].max()
        min_value = dfs_final.loc[similar_region].min()
        max_index = dfs_final.loc[similar_region].idxmax()
        min_index = dfs_final.loc[similar_region].idxmin()
        
        plt.scatter(max_index, max_value, color=colors[i % len(colors)], s=100, zorder=10, edgecolor='black', marker='o')
        plt.scatter(min_index, min_value, color=colors[i % len(colors)], s=100, zorder=10, edgecolor='black', marker='^')

    max_value_target = dfs_final.loc[region_name].max()
    min_value_target = dfs_final.loc[region_name].min()
    max_index_target = dfs_final.loc[region_name].idxmax()
    min_index_target = dfs_final.loc[region_name].idxmin()
    plt.scatter(max_index_target, max_value_target, color='orange', s=100, zorder=10, edgecolor='black', marker='o', label='Максимум')
    plt.scatter(min_index_target, min_value_target, color='orange', s=100, zorder=10, edgecolor='black', marker='^', label='Минимум')
    plt.title(f"{domen}. Динамика индексов для региона '{region_name}' и {k} похожих регионов", fontsize=18, fontweight='bold', color='#333')
    plt.xlabel("Год", fontsize=14)
    plt.ylabel("Индекс", fontsize=14)
    
    plt.legend(fontsize=12, loc="best")
    plt.xticks(rotation=45)
    
    plt.tick_params(axis='both', which='major', labelsize=12)
    
    plt.tight_layout()
    plt.show()

def plot_top_successful_regions_dynamic(dfs_final, domen, top_n=10, alpha=0.5):
    """
    Строит горизонтальную диаграмму топ-N самых успешных регионов, учитывая уровень индекса за последний год и динамику.
    Добавлены декоративные элементы для улучшения визуализации.
    
    Args:
        dfs_final (pd.DataFrame): Датафрейм с индексами регионов (индексы - регионы, колонки - годы).
        top_n (int): Количество самых успешных регионов для отображения.
        alpha (float): Вес для динамики (0 <= alpha <= 1). 
                       Вес уровня последнего года = 1 - alpha.
    """
    if not (0 <= alpha <= 1):
        raise ValueError("Параметр alpha должен быть в диапазоне от 0 до 1.")
    
    index_growth = dfs_final.diff(axis=1).mean(axis=1)
    last_year = dfs_final.columns[-1]
    last_year_index = dfs_final[last_year]
    combined_score = alpha * index_growth + (1 - alpha) * last_year_index
    top_regions = combined_score.sort_values(ascending=False).head(top_n)
    norm = plt.Normalize(min(top_regions), max(top_regions))
    cmap = matplotlib.colormaps['coolwarm']
    
    fig, ax = plt.subplots(figsize=(12, 7))  # Создаем ось ax
    bars = ax.barh(top_regions.sort_values().index, top_regions.sort_values(), color=cmap(norm(top_regions.sort_values())))
    
    for bar in bars:
        bar.set_edgecolor('black')
        bar.set_linewidth(1.5) 
        for i, (region, value) in enumerate(zip(top_regions.sort_values().index, top_regions.sort_values())):
            ax.text(value + 0.01, i, f"({value:.3f})", va='center', fontsize=12, color='black', weight='bold')
    
    ax.set_title(f"{domen}. Топ-{top_n} самых успешных регионов с учетом динамики ({last_year})", fontsize=16, weight='bold')
    ax.set_xlabel("Куммулятивный рейтинг", fontsize=14)
    ax.set_ylabel("Регионы", fontsize=14)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

def plot_bottom_successful_regions_dynamic(dfs_final, domen, bottom_n=10, alpha=0.5):
    """
    Строит горизонтальную диаграмму bottom-N наименее успешных регионов, учитывая уровень индекса за последний год и динамику.
    Добавлены декоративные элементы для улучшения визуализации.
    
    Args:
        dfs_final (pd.DataFrame): Датафрейм с индексами регионов (индексы - регионы, колонки - годы).
        bottom_n (int): Количество наименее успешных регионов для отображения.
        alpha (float): Вес для динамики (0 <= alpha <= 1). 
                       Вес уровня последнего года = 1 - alpha.
    """
    if not (0 <= alpha <= 1):
        raise ValueError("Параметр alpha должен быть в диапазоне от 0 до 1.")
    
    index_growth = dfs_final.diff(axis=1).mean(axis=1)
    last_year = dfs_final.columns[-1]
    last_year_index = dfs_final[last_year]
    combined_score = alpha * index_growth + (1 - alpha) * last_year_index
    
    bottom_regions = combined_score.sort_values(ascending=True).head(bottom_n)
    norm = plt.Normalize(min(bottom_regions), max(bottom_regions))
    cmap = matplotlib.colormaps['Reds']  # Используем цветовую схему для негативных результатов
    
    fig, ax = plt.subplots(figsize=(12, 7))
    bars = ax.barh(bottom_regions.sort_values(ascending=False).index, 
                   bottom_regions.sort_values(ascending=False), 
                   color=cmap(norm(bottom_regions.sort_values(ascending=False))))
    
    for bar in bars:
        bar.set_edgecolor('black')
        bar.set_linewidth(1.5) 
        for i, (region, value) in enumerate(zip(bottom_regions.sort_values(ascending=False).index, 
                                                bottom_regions.sort_values(ascending=False))):
            ax.text(value + 0.01, i, f"({value:.3f})", va='center', fontsize=12, color='black', weight='bold')
    
    ax.set_title(f"{domen}. Топ-{bottom_n} наименее успешных регионов с учетом динамики ({last_year})", 
                 fontsize=16, weight='bold', color='darkred')
    ax.set_xlabel("Кумулятивный рейтинг", fontsize=14)
    ax.set_ylabel("Регионы", fontsize=14)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


def mean_weight(weights_by_base, weights_by_current, alpha=0.9):
    return weights_by_base * alpha + (1 - alpha) * weights_by_current

def count_impact_per_component(df : pd.DataFrame, weights : np.array):
    '''
    Функция принимает два аргумента:
        df : текущий датафрейм
        weights : веса для признаков
    Return:
        Умножаем веса на значения признаков
    '''
    buff_df = df.copy()
    buff_df.iloc[:,3:-1] = buff_df.iloc[:,3:-1] * weights 
    return buff_df

def combine_for_weights_and_importances(dfs1, dfs2, index_share):
    '''
    Функция принимает два датафрейма и коэффициент сглаживания
    Нужна, чтобы вычислять важность признака в структуре финального индекса
    Возвращает важность каждого признака в долях
    '''
    dfs_final = {}
    for year in dfs1.keys():
        dfs_final[year] = dfs1[year].copy()
        dfs_final[year].iloc[:,3:-1] = dfs_final[year].iloc[:,3:-1] * index_share + dfs2[year].iloc[:,3:-1] * (1-index_share)
        total_sum = dfs_final[year].iloc[:,3:-1].sum(axis=1)
        dfs_final[year] = dfs_final[year].drop('index', axis=1)
        for col in dfs_final[year].columns[3:]:
            dfs_final[year][col] /= total_sum
    return dfs_final

def rank_importances(clear_domen, domen, importances_of_domen, region, year):
    if isinstance(year, list):
        start = year[0]
        end = year[1]
        df = importances_of_domen[start][importances_of_domen[start]['object_name'] == region].iloc[:,3:].transpose()
        df.columns = ['Влияние на ИРР']
        for year in range(start+1, end+1, 1):
            chosen = importances_of_domen[year][importances_of_domen[year]['object_name'] == region].iloc[:,3:].transpose()
            chosen.columns = ['Влияние на ИРР']
            df += chosen
        df['Влияние на ИРР'] = df['Влияние на ИРР'] / (end - start + 1)
        df = df.sort_values(by='Влияние на ИРР', ascending=False) 
        return df
    else:   
        chosen = importances_of_domen[year][importances_of_domen[year]['object_name'] == region].iloc[:,3:].transpose()
        chosen.columns = ['Влияние на ИРР']
        values = clear_domen[year][clear_domen[year]['object_name'] == region].iloc[:, 3:].transpose()
        values.columns = ['Значение показателя']
        chosen = pd.concat([chosen, values], axis=1)
        chosen = chosen.sort_values(by='Влияние на ИРР', ascending=False)

        category = {'Положительно' : [], 'Отрицательно' : []}
        clear = clear_domen[year][clear_domen[year]['object_name'] == region].iloc[:,3:]
        modified = domen[year][domen[year]['object_name'] == region].iloc[:,3:]

        for col in clear.columns:
            if clear[col].values == modified[col].values:
                category['Положительно'].append(col)
            else:
                category['Отрицательно'].append(col)
        chosen['Тип показателя'] = chosen.index.to_series().apply(
            lambda x: 'Стимулянт' if x in category['Положительно'] else 'Дестимулянт'
        )

        # гарантия нормировки
        total = chosen['Влияние на ИРР'].sum()
        chosen['Влияние на ИРР'] = np.abs(chosen['Влияние на ИРР']) / total
        chosen = chosen.sort_values(by='Влияние на ИРР', ascending=False)
        
        return chosen

def ravel_domen_final(data):
    '''
    data: pd.DataFrame
    Вытягиевает в формате столбцов: год, регион, значение индекса
    '''
    data = data.reset_index()
    melted_data = data.melt(id_vars=[data.columns[0]], 
                             var_name='год', 
                             value_name='значение индекса')
    
    melted_data.rename(columns={data.columns[0]: 'регион'}, inplace=True)
    
    return melted_data

def ravel_domen_dict(data_dict):
    '''
    data_dict: dict
    Словарь, где ключи - годы, значения - DataFrame с колонками: object_name, object_level, year и другие признаки.
    
    Возвращает объединенный DataFrame с колонками: год, object_name, object_level и остальные признаки.
    '''
    df_list = []
    
    for year, df in data_dict.items():
        if df.empty:
            raise ValueError(f"DataFrame for year {year} is empty.")
        if not all(col in df.columns for col in ['object_name', 'object_level', 'year']):
            raise ValueError(f"DataFrame for year {year} is missing required columns.")
        
        df['year'] = year
        
        df_list.append(df)
    
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df

def analyze_indicator(df, agg_func="mean", region=None):
    """
    Интерактивно анализирует динамику показателя по годам, с возможностью фильтрации по региону.

    :param df: Dict[int : pd.DataFrame]
    :param agg_func: "mean" (по умолчанию) или "median" - метод агрегирования
    :param region: Название региона (или None, чтобы смотреть по всем)
    """
    # 1️⃣ Фильтрация по региону, если указан
    df = ravel_domen_dict(df)
    if region:
        df = df[df["object_name"] == region]

    # 2️⃣ Выбор показателя
    indicators = df.columns[3:]  # Все столбцы после 'year'
    print("\nВыберите показатель для анализа:")
    for i, col in enumerate(indicators, 1):
        print(f"{i}) {col}")

    choice = int(input("\nВведите номер показателя: ")) - 1
    indicator = indicators[choice]

    # 3️⃣ Агрегация данных (по годам)
    if agg_func == "mean":
        df_grouped = df.groupby("year")[indicator].mean()
    elif agg_func == "median":
        df_grouped = df.groupby("year")[indicator].median()
    else:
        raise ValueError("Неверное значение agg_func. Используйте 'mean' или 'median'.")

    # 4️⃣ Расчет стандартного отклонения для доверительного интервала
    df_std = df.groupby("year")[indicator].std()

    # 5️⃣ Автоматический тренд (линейная регрессия)
    years = df_grouped.index.values
    y_values = df_grouped.values

    # Выполнение линейной регрессии
    slope, intercept, r_value, p_value, std_err = stats.linregress(years, y_values)

    # Проверка значимости коэффициента наклона
    if p_value < 0.05:  # Уровень значимости 0.05
        trend_slope = slope
        if trend_slope > 0:
            trend = "Восходящий 📈"
        elif trend_slope < 0:
            trend = "Нисходящий 📉"
    else:
        trend = "Стационарный ➖"

    region_str = f" ({region})" if region else " (по всем регионам)"
    print(f"\n📊 Тренд показателя '{indicator}'{region_str}: {trend}")

    # 6️⃣ Темпы роста (% изменения от прошлого года)
    df_pct_change = df_grouped.pct_change() * 100

    # 7️⃣ Топ-3 года по значению
    top_years = df_grouped.nlargest(3)
    print("\n🏆 Топ-3 года по показателю:")
    print(top_years)

    # 🔹 Визуализация 🔹
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))

    # --- Первый график: Динамика показателя ---
    axes[0].plot(df_grouped.index, df_grouped.values, marker="o", label="Динамика показателя", color="b")
    axes[0].plot(years, intercept + slope * years, linestyle="dashed", color="red", label="Линия тренда")
    axes[0].fill_between(df_grouped.index, df_grouped - df_std, df_grouped + df_std, color="gray", alpha=0.2)

    axes[0].set_xlabel("Год")
    # axes[0].set_title(f"Динамика показателя: {indicator}{region_str}")
    axes[0].legend()
    axes[0].grid(True)

    # --- Второй график: Темпы роста (столбиками) ---
    colors = ["green" if val >= 0 else "red" for val in df_pct_change.values]

    bars = axes[1].bar(df_pct_change.index, df_pct_change.values, color=colors, alpha=0.7)

    # 🔢 Добавление значений над столбцами
    for bar in bars:
        height = bar.get_height()
        if not np.isnan(height):  # Проверка, чтобы не писать NaN
            axes[1].text(bar.get_x() + bar.get_width()/2, height, f"{height:.1f}%", 
                         ha="center", va="bottom" if height > 0 else "top", fontsize=10, color="black")

    axes[1].set_xlabel("Год")
    axes[1].set_ylabel("Темпы прироста (%)")
    axes[1].set_title(f"Темпы прироста показателя по годам{region_str}")
    axes[1].axhline(0, color="gray", linestyle="dashed")  # Горизонтальная линия на 0%
    axes[1].grid(True)

    plt.tight_layout()

def analyze_indicator_interactive(df):
    """
    Интерактивная версия функции analyze_indicator с выбором региона и показателя.
    """
    df = ravel_domen_dict(df)  # Преобразование данных

    # Виджет для выбора региона
    region_widget = widgets.Dropdown(
        options=[None] + list(df["object_name"].unique()),  # None = все регионы
        value=None,
        description="Регион:"
    )

    # Виджет для выбора метода агрегирования
    agg_widget = widgets.RadioButtons(
        options=["mean", "median"],
        value="mean",
        description="Агрегация:"
    )

    # Виджет для выбора показателя (обновляется динамически)
    indicator_widget = widgets.Dropdown(
        options=df.columns[3:], 
        description="Показатель:"
    )

    # Функция обработки
    def process(region, agg_func, indicator):
        df_filtered = df[df["object_name"] == region] if region else df
        
        # Агрегация данных
        if agg_func == "mean":
            df_grouped = df_filtered.groupby("year")[indicator].mean()
        else:
            df_grouped = df_filtered.groupby("year")[indicator].median()
        
        # Стандартное отклонение
        df_std = df_filtered.groupby("year")[indicator].std()

        # Линейная регрессия
        years = df_grouped.index.values
        y_values = df_grouped.values
        slope, intercept, r_value, p_value, std_err = stats.linregress(years, y_values)
        
        trend = "Стационарный ➖"
        if p_value < 0.05:
            if slope > 0:
                trend = "Восходящий 📈"
            elif slope < 0:
                trend = "Нисходящий 📉"

        print(f"\n📊 Тренд показателя '{indicator}' ({region if region else 'по всем регионам'}): {trend}")

        # Темпы роста
        df_pct_change = df_grouped.pct_change() * 100

        # Визуализация
        fig, axes = plt.subplots(2, 1, figsize=(12, 10))

        # --- График динамики ---
        axes[0].plot(df_grouped.index, df_grouped.values, marker="o", label="Динамика", color="b")
        axes[0].plot(years, intercept + slope * years, linestyle="dashed", color="red", label="Тренд")
        axes[0].fill_between(df_grouped.index, df_grouped - df_std, df_grouped + df_std, color="gray", alpha=0.2)
        axes[0].legend()
        axes[0].grid(True)

        # --- График темпов роста ---
        colors = ["green" if val >= 0 else "red" for val in df_pct_change.values]
        bars = axes[1].bar(df_pct_change.index, df_pct_change.values, color=colors, alpha=0.7)

        for bar in bars:
            height = bar.get_height()
            if not np.isnan(height):
                axes[1].text(bar.get_x() + bar.get_width()/2, height, f"{height:.1f}%", 
                             ha="center", va="bottom" if height > 0 else "top", fontsize=10)

        axes[1].axhline(0, color="gray", linestyle="dashed")
        axes[1].grid(True)

        plt.tight_layout()
        plt.show()

    # Создание интерактивного интерфейса
    interactive_plot = interactive(process, 
                                   region=region_widget, 
                                   agg_func=agg_widget, 
                                   indicator=indicator_widget)
    
    display(interactive_plot)

In [50]:
neutral = [
    'object_name',
    'object_level',
    'year',
]

stimulants = [
    'Коэффициенты естественного прироста населения на 1000 человек населения (ND)',
    'Суммарный коэффициент рождаемости (число детей на 1 женщину)',
    # 'Ожидаемая продолжительность жизни при рождении: Все население (число лет)',
    'Ожидаемая продолжительность жизни при рождении: Женщины (число лет)',
    'Ожидаемая продолжительность жизни при рождении: Мужчины (число лет)',
    'Удельный вес городского населения в общей численности населения (оценка на конец года, в процентах)',
]

destimulants = [
    'Коэффициенты демографической нагрузки: Всего (оценка на конец года, на 1000 человек трудоспособного возраста приходится лиц нетрудоспособных возрастов)',
    'Смертность населения в трудоспособном возрасте (число умерших на 100 000 человек соответствующего возраста)',
    'Соотношение браков и разводов (на 1000 браков приходится разводов)',
    'Коэффициенты младенческой смертности (число детей, умерших в возрасте до 1 года, на 1000 родившихся живыми)',
    'Общие коэффициенты смертности (число умерших на 1000 человек населения)',
]

others = [
    
]

# нейтральные добавляются как есть
population = {year : Население[year][neutral].copy() for year in Население.keys()}

#########################
clear_population = {year : Население[year][neutral].copy() for year in Население.keys()}
for year, data in clear_population.items():
    for stim in stimulants:
        data[stim] = Население[year][stim]
for year, data in clear_population.items():
    for destim in destimulants:
        data[destim] = Население[year][destim]
##########################

# стимулянты добавляются как есть
for year, data in population.items():
    for stim in stimulants:
        data[stim] = Население[year][stim]

# дестимулянты инвертируется
for year, data in population.items():
    for destim in destimulants:
        data[destim] = -Население[year][destim]

# другие пока не учитываются

In [52]:
neutral = [
    'object_name',
    'object_level',
    'year'
]

stimulants = [
    'Валовой коэффициент охвата дошкольным образованием (на конец года, в процентах от численности детей в возрасте 1‒6 лет)',
    'Выпуск бакалавров, специалистов, магистров (тысяч человек)',
    # 'Выпуск квалифицированных рабочих и служащих (тысяч человек)',
    'Выпуск обучающихся организациями, осуществляющих образовательную деятельность по образовательным программам начального, основного и среднего общего образования: Выпуск обучающихся с аттестатом о среднем общем образовании (тысяч человек)',
    'Выпуск обучающихся организациями, осуществляющих образовательную деятельность по образовательным программам начального, основного и среднего общего образования: Выпуск обучающихся с аттестатом об основном общем образовании (тысяч человек)',
    # 'Выпуск специалистов среднего звена (тысяч человек)',
    # 'Обеспеченность детей дошкольного возраста местами в организациях, осуществляющих образовательную деятельность по образовательным программам дошкольного образования, присмотр и уход за детьми (на конец года, приходится мест на 1000 детей)',
    'Прием на обучение по программам бакалавриата, специалитета, магистратуры (тысяч человек)',
    # 'Прием на обучение по программам подготовки квалифицированных рабочих, служащих (тысяч человек)',
    'Прием на обучение по программам подготовки специалистов среднего звена (тысяч человек)',
    'Численность аспирантов (на конец года, человек)',
    # 'Численность воспитанников организаций, осуществляющих образовательную деятельность по образовательным программам дошкольного образования, присмотр и уход за детьми (на конец года, тысяч человек)',
    'Численность докторантов (на конец года, человек)',
    'Численность обучающихся организаций, осуществляющих образовательную деятельность по образовательным программам начального, основного и среднего общего образования (на начало учебного года, тысяч человек)',
    # 'Численность студентов, обучающихся по программам бакалавриата, специалитета, магистратуры (на начало учебного года, тысяч человек)',
    # 'Численность студентов, обучающихся по программам бакалавриата, специалитета, магистратуры на 10 000 человек населения (на начало учебного года, человек)',
    'Численность студентов, обучающихся по программам подготовки квалифицированных рабочих, служащих (на конец года,тысяч человек)',
    'Численность студентов, обучающихся по программам подготовки специалистов среднего звена (на начало учебного года, тысяч человек)'
]

destimulants = [
    'Удельный вес обучающихся во вторую и третью смены в организациях, осуществляющих образовательную деятельность по образовательным программам начального, основного и среднего общего образования (на начало учебного года, в процентах от общей численности обучающихся)'
]

others = [
    # соотношение приема и выпуска
]

# нейтральные добавляются как есть
education = {year : Образование[year][neutral].copy() for year in Образование.keys()}

#########################
clear_education = {year : Образование[year][neutral].copy() for year in Образование.keys()}
for year, data in clear_education.items():
    for stim in stimulants:
        data[stim] = Образование[year][stim]
for year, data in clear_education.items():
    for destim in destimulants:
        data[destim] = Образование[year][destim]
##########################

# стимулянты добавляются как есть
for year, data in education.items():
    for stim in stimulants:
        data[stim] = Образование[year][stim]
    
    

# дестимулянты инвертируется
for year, data in education.items():
    for destim in destimulants:
        data[destim] = -Образование[year][destim]

# другие пока не учитываются

In [56]:
neutral = [
    'object_name',
    'object_level',
    'year'
]

# Стимулянты — показатели, которые должны увеличиваться для улучшения ситуации
stimulants = [
    'Изменение среднегодовой численности занятых (в процентах к предыдущему году)',
    'Нагрузка незанятого населения, состоящего на регистрационном учете в органах службы занятости населения, в расчете на одну заявленную вакансию (на конец года, человек)',
    'Потребность в работниках, заявленная работодателями в органы службы занятости населения (на конец года, человек)',
    'Среднегодовая численность занятых (тысяч человек)',
    'Уровень занятости населения (по данным выборочных обследований рабочей силы, в процентах)',
    'Уровень занятости населения в трудоспособном возрасте (по данным выборочных обследований рабочей силы, в процентах)',
    'Уровень участия в составе рабочей силы (по данным выборочных обследований рабочей силы, в процентах)',
    'Численность рабочей силы (по данным выборочных обследований рабочей силы, тысяч человек)'
]

# Дестимулянты — показатели, которые должны уменьшаться для улучшения ситуации
destimulants = [
    'Уровень безработицы: Уровень безработицы (по данным выборочных обследований рабочей силы, в процентах)',
    'Уровень безработицы: Уровень безработицы в трудоспособном возрасте (по данным выборочных обследований рабочей силы, в процентах)',
    'Уровень безработицы: Уровень зарегистрированной безработицы (на конец года, в процентах)',
    'Численность безработных (по данным выборочных обследований рабочей силы, тысяч человек)',
    'Численность зарегистрированных безработных (на конец года, тысяч человек)',
    'Численность незанятых граждан, состоящих на учете в органах службы занятости населения в целях поиска подходящей работы (на конец года, тысяч человек)'
]

others = [
    # Здесь можно добавить другие переменные, если нужно
]

# Нейтральные данные добавляются как есть
labor = {year: Труд[year][neutral].copy() for year in Труд.keys()}

#########################
clear_labor = {year: Труд[year][neutral].copy() for year in Труд.keys()}
for year, data in clear_labor.items():
    for stim in stimulants:
        data[stim] = Труд[year][stim]
for year, data in clear_labor.items():
    for destim in destimulants:
        data[destim] = Труд[year][destim]
##########################

# Стимулянты добавляются как есть
for year, data in labor.items():
    for stim in stimulants:
        data[stim] = Труд[year][stim]
    
# Дестимулянты инвертируются
for year, data in labor.items():
    for destim in destimulants:
        data[destim] = -Труд[year][destim]

# Другие переменные пока не учитываются


In [60]:
# оставляем только регионы
population = {year : population[year][population[year]['object_level'] == 'регион'] for year in population.keys()}
clear_population = {year : clear_population[year][clear_population[year]['object_level'] == 'регион'] for year in clear_population.keys()}

education = {year : education[year][education[year]['object_level'] == 'регион'] for year in education.keys()}
clear_education = {year : clear_education[year][clear_education[year]['object_level'] == 'регион'] for year in clear_education.keys()}

In [62]:
for year, df in population.items():
    population[year] = drop_missing_values(df, population[list(population.keys())[0]].columns)
for year, df in clear_population.items():
    clear_population[year] = drop_missing_values(df, clear_population[list(clear_population.keys())[0]].columns)


for year, df in education.items():
    education[year] = drop_missing_values(df, education[list(education.keys())[0]].columns)
for year, df in clear_education.items():
    clear_education[year] = drop_missing_values(df, clear_education[list(clear_education.keys())[0]].columns)

In [64]:
# def save_hdf5(data: dict[int, pd.DataFrame], filename: str):
#     with pd.HDFStore(filename, mode="w") as store:
#         for year, df in data.items():
#             store.put(f"year_{year}", df)

# def load_hdf5(filename: str) -> dict[int, pd.DataFrame]:
#     data = {}
#     with pd.HDFStore(filename, mode="r") as store:
#         for key in store.keys():
#             year = int(key.split("_")[1])  # Извлекаем год из ключа
#             data[year] = store[key]

#     return data

In [66]:
domen_mapping = {
    "Население": ("population", "clear_population"),
    "Образование": ("education", "clear_education"),
}

# Виджет для выбора домена
domen_widget = widgets.Dropdown(
    options=domen_mapping.keys(),
    description='Домен:'
)

# Виджеты для выбора параметров
region_widget = widgets.Dropdown(
    options=[],  # Будет заполняться при выборе домена
    description='Регион:'
)

method_calculation_widget = widgets.Dropdown(
    options = ['PCA', 'Обратная дисперсия', 'Равные веса'],
    value='PCA',  # Значение по умолчанию - PCA
    description='Метод расчета:'
)

base_year_widget = widgets.IntSlider(
    value=2010,
    min=2000,
    max=2022,
    step=1,
    description='Базовый год:'
)

index_share_widget = widgets.FloatSlider(
    value=0.9,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Alpha:'
)

# Кнопки управления
load_domen_button = widgets.Button(description="Загрузить Домен", button_style='primary')
interactive_analysis_button = widgets.Button(description="Интерактивный Анализ", button_style='info')
update_button = widgets.Button(description="Обновить данные", button_style='success')
show_table_button = widgets.Button(description="Показать таблицу", button_style='success')

# Поля вывода
output_domen = widgets.Output()
output_interactive = widgets.Output()
output_main = widgets.Output()
output_table = widgets.Output()

# Глобальные переменные
domen = None
clear_domen = None
domen_name = ""
region_name = ""
method_calculation = 1

# Функция выбора домена
def load_domen(_=None):
    global domen, clear_domen, domen_name, region_name
    domen_key = domen_widget.value
    domen_var, clear_domen_var = domen_mapping[domen_key]  # Получаем переменные

    # Устанавливаем глобальные переменные
    domen = globals()[domen_var]
    clear_domen = globals()[clear_domen_var]
    domen_name = domen_key  # Человекочитаемое имя домена

    # Обновляем список регионов
    region_widget.options = domen[list(domen.keys())[0]]['object_name'].unique()

    output_domen.clear_output()
    with output_domen:
        print(f"Выбран домен: {domen_name}")

load_domen_button.on_click(load_domen)

# Функция для интерактивного анализа
def interactive_analysis(_=None):
    global clear_domen
    output_interactive.clear_output()
    with output_interactive:
        analyze_indicator_interactive(clear_domen)

interactive_analysis_button.on_click(interactive_analysis)

# Основная обработка данных
def process_data(_=None):
    global domen, clear_domen, domen_name, region_name, method_calculation

    if domen is None:
        return
    region_name = region_widget.value
    base_year = base_year_widget.value
    index_share = index_share_widget.value
    calculation_method = {'PCA' : 1, 'Обратная дисперсия' : 2, 'Равные веса' : 3}
    method_calculation = calculation_method[method_calculation_widget.value]

    output_main.clear_output()

    with output_main:
        print(f"Обновление данных для региона: {region_name}, базовый год: {base_year}, индекс: {index_share}")

        # Фильтрация данных по региону
        domen_filtered = {year: domen[year][domen[year]['object_level'] == 'регион'] for year in domen.keys()}

        # Удаление пропусков
        for year, df in domen_filtered.items():
            domen_filtered[year] = drop_missing_values(df, domen_filtered[list(domen_filtered.keys())[0]].columns)

        # Нормализация по текущему году и базовому году
        domen_norm_by_current = normalized_dict_by_minmax(domen_filtered)
        domen_norm_by_base = normalized_dict_by_minmax_by_base_year(domen_filtered, base_year)

        # Расчет индексов и весов
        domen_indexes_by_base, weights_by_base = calculate_index_with_weights(domen_norm_by_base, method_calculation)
        domen_indexes_by_current, weights_by_current = calculate_index_with_weights(domen_norm_by_current, method_calculation)

        # Финальный индекс
        domen_final = combine_indices(domen_indexes_by_base, domen_indexes_by_current, index_share)

        # Построение графиков
        plot_index_trends_multi(domen_indexes_by_base, domen_indexes_by_current, domen_final, region_name, domen_name)
        plot_index_trends_with_similar(domen_indexes_by_base, domen_indexes_by_current, domen_final, region_name, domen_name, 3)
        plot_top_successful_regions_dynamic(domen_final, domen_name, 15, 0.1)
        plot_bottom_successful_regions_dynamic(domen_final, domen_name, 15, 0.1)

        # Оценка влияния компонентов
        domen_norm_base_weighted = {year: count_impact_per_component(domen_norm_by_base[year], weights_by_base) for year in domen_norm_by_base.keys()}
        domen_norm_current_weighted = {year: count_impact_per_component(domen_norm_by_current[year], weights_by_current) for year in domen_norm_by_current.keys()}
        global importances_of_domen
        importances_of_domen = combine_for_weights_and_importances(domen_norm_base_weighted, domen_norm_current_weighted, index_share)

update_button.on_click(process_data)

year_widget = widgets.IntSlider(
    value=2000,
    min=2000,
    max=2022,
    step=1,
    description='Год:'
)

def show_table(_=None):
    # Очищаем вывод графиков
    output_main.clear_output()

    output_table.clear_output()
    with output_table:
        selected_year = year_widget.value  # Получаем выбранный год
        print(f"Индекс региона: {region_widget.value}, Год: {selected_year}")  # Выводим индекс региона и выбранный год
        df_importances = rank_importances(clear_domen, domen, importances_of_domen, region_widget.value, selected_year)
        display(df_importances)

show_table_button.on_click(show_table)

clear_screen_button = widgets.Button(description="Очистить экран", button_style='danger')

# Функция очистки экрана
def clear_screen(_=None):
    # Очищаем все области вывода
    output_domen.clear_output()
    output_interactive.clear_output()
    output_main.clear_output()
    output_table.clear_output()

clear_screen_button.on_click(clear_screen)

# Отображение интерфейса с кнопкой очистки экрана
display(VBox([
    HBox([domen_widget, load_domen_button]),
    output_domen,
    interactive_analysis_button,
    output_interactive,
    HBox([region_widget, base_year_widget, index_share_widget]),
    HBox([update_button, method_calculation_widget]),
    show_table_button,
    year_widget,  # Новый виджет для выбора года
    output_main,
    output_table,
    clear_screen_button  # Добавляем кнопку очистки экрана
]))